# Práctica 0 
### Adrian Carlos Skaczylo e Itziar López Almagro



In [29]:
import numpy as np
import random
N = 1000

In [50]:
def f(x):
     return -(x)**2 +1


In [42]:
def maximo_relativos(a,b,fun):
    subintervalos = np.linspace(a,b,N) #Division del intervalo [a,b]

    valores = [fun(x) for x in subintervalos] #evaluamos f(x_i)
    return max(valores)


In [52]:
def integra_mc(fun, a, b, num_puntos =10000):

    
    #Calcular maximo M ¿Como calcular maximo de la funcion ?
    M = maximo_relativos(a,b,fun)
    N_Debajo = 0
    for i in range(0,num_puntos):
        
        #generar un punto
        punto = (random.uniform(a,b),random.uniform(0,M))

        if punto[1] <= fun(punto[0]) :
            N_Debajo +=1

    return (N_Debajo/num_puntos)*(b-a)*M


In [ ]:
def integra_mc2(fun,a,b,num_puntos=10000):
    M = maximo_relativos(a,b,fun)
    x_vector = np.linspace(a,b,num_puntos)
    y_vector = np.linspace(0,M,num_puntos)
    f_evaluados = f(x_vector)
    
    N_debajo = np.sum(y_vector <= f_evaluados)

    return (N_debajo/num_puntos)*(b-a)*M

In [ ]:
integra_mc(f,-1,1,10000)

In [53]:
integra_mc2(f,-1,1,10000)

0.999998997996996


np.float64(0.00019999979959939922)